In [1]:
import numpy as np
import numba

In [2]:
@numba.njit
def _random_mutation_generator(sequence, rate, number_fixed):
    if number_fixed:
        num_mutations = int(rate*len(sequence))
    else:
        num_mutations = np.random.poisson(len(sequence) * rate)
    positions = np.random.choice(np.arange(len(sequence)), num_mutations, replace=False)
    return positions

In [3]:
testseq = 'A' * 100

In [4]:
_random_mutation_generator(testseq, rate=0.1, number_fixed=True)

array([68, 89, 64, 59, 76, 48,  4, 70, 29, 51])

In [5]:
def random_mutation_generator(sequence, rate, num_mutants, number_fixed):
    mutant_list = np.empty(num_mutants, dtype='object')
    for i in range(num_mutants):
        mutant_list[i] = _random_mutation_generator(sequence, rate, number_fixed)
    
    return mutant_list

In [6]:
random_mutation_generator(testseq, rate=0.1, num_mutants=10, number_fixed=True)

array([array([58, 64, 74, 29, 99, 31, 82,  6, 13, 41]),
       array([95, 43, 70, 65, 86, 45, 92, 99, 97, 50]),
       array([37, 75, 83, 80,  6, 74, 61, 92, 69, 39]),
       array([30, 34, 77, 99, 15, 28, 70, 17, 89, 23]),
       array([51, 83, 61, 20, 44, 49, 27, 14, 36, 64]),
       array([79, 80, 76, 55, 81, 33, 62,  8, 88, 73]),
       array([65, 21, 24, 98, 70, 36,  0, 99, 33, 47]),
       array([32, 50,  1, 57, 34, 62,  8, 17, 14, 25]),
       array([63, 94, 86, 91, 65, 48, 74,  0, 56,  7]),
       array([ 5, 22, 28, 76, 17, 45, 66, 82,  0, 20])], dtype=object)

In [7]:
@numba.njit
def filter_letter(x, letter):
    return x != letter


@numba.njit
def filter_mutation(letter, alph):
    j = 0
    for i in range(alph.size):
        if filter_letter(alph[i], letter):
            j += 1
    result = np.empty(j, dtype=alph.dtype)
    j = 0
    for i in range(alph.size):
        if filter_letter(alph[i], letter):
            result[j] = alph[i]
            j += 1
    return result

def mutate_from_index(sequence, index, allowed_alph):
    seq_list = list(sequence)
    for loci, mutation in index:
        seq_list[loci] = filter_mutation(seq_list[loci], allowed_alph)[mutation].lower()
    return "".join(seq_list)

In [58]:
@numba.njit
def choose_mutation_bias(letter, letters, allowed_alph):
    if letter == 'A':
        _alph = allowed_alph[0]
    elif letter == 'C':
        _alph = allowed_alph[1]
    elif letter == 'G':
        _alph = allowed_alph[2]
    elif letter == 'T':
        _alph = allowed_alph[3]
    
    alph = np.empty(np.sum(_alph), dtype=letters.dtype)
    i = 0
    j = 0
    for x in _alph:
        if x:
            alph[i] = letters[j]
            i += 1
        j += 1

    return alph

@numba.njit
def make_mutation(letter, letters, allowed_alph):
    alph = choose_mutation_bias(letter, letters, allowed_alph)
    index = np.random.choice(np.arange(len(alph)))
    return alph[index]

def mutate_with_bias(sequence, index, letters, allowed_alph):
    seq_list = list(sequence)
    for loci in index:
        seq_list[loci] = make_mutation(seq_list[loci], letters, allowed_alph).lower()
    return "".join(seq_list)

In [59]:
allowed_alph = np.array([[False, False, False, True],
                         [False, False, True, False],
                         [False, True, False, False],
                         [True, False, False, False]])
letters = np.array(["A", "C", "G", "T"])

mutate_with_bias(testseq, [10], letters, allowed_alph)

'AAAAAAAAAAtAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'

In [60]:
allowed_alph = np.array([[False, True, False, True],
                         [False, False, True, False],
                         [False, True, False, False],
                         [True, False, False, False]])
letters = np.array(["A", "C", "G", "T"])

mutate_with_bias(testseq, [10, 20, 30, 40, 50, 60, 70, 80, 90], letters, allowed_alph)

'AAAAAAAAAAtAAAAAAAAAcAAAAAAAAAtAAAAAAAAAtAAAAAAAAAtAAAAAAAAAcAAAAAAAAAtAAAAAAAAAtAAAAAAAAAtAAAAAAAAA'

In [61]:
def mutations_rand(
    sequence, 
    num_mutants,
    rate,
    site_start=0, 
    site_end=None, 
    alph_type="DNA",
    allowed_alph=None,
    number_fixed=False,
    keep_wildtype=False,
):
    
    # Get site to mutate
    
    if site_end==None:
        mutation_window = sequence[site_start:]
    else:
        mutation_window = sequence[site_start:site_end]
    
    # Create list
    if keep_wildtype:
        mutants = np.empty(num_mutants+1, dtype='U{}'.format(len(sequence)))
        mutants[0] = sequence
        i_0 = 1
    else:
        mutants = np.empty(num_mutants, dtype='U{}'.format(len(sequence)))
        i_0 = 0
        
    mutant_indeces = random_mutation_generator(mutation_window, rate, num_mutants, number_fixed)
    if alph_type == "DNA":
        letters = np.array(["A", "C", "G", "T"])
    elif alph_type == "Numeric":
        letters = np.arange(4)
    else:
        raise ValueError("Alphabet type has to be either \"DNA\" or \"Numeric\"")
    
    if allowed_alph is not None:
        for i, x in enumerate(mutant_indeces):
            mutants[i + i_0] = sequence[0:site_start] + mutate_with_bias(mutation_window, x, letters, allowed_alph) + sequence[site_end:]
    else:
        for i, x in enumerate(mutant_indeces):
            mutants[i + i_0] = sequence[0:site_start] + mutate_from_index(mutation_window, x, letters) + sequence[site_end:]
        
    return mutants

In [62]:
allowed_alph = np.array([[False, False, False, True],
                         [False, False, True, False],
                         [False, True, False, False],
                         [True, False, False, False]])

mutations_rand(testseq, 10, rate=0.1, site_start=10, site_end=90, 
    alph_type="DNA",
    allowed_alph=allowed_alph,
    number_fixed=True,
    keep_wildtype=True,
)

array(['AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA',
       'AAAAAAAAAAAAtAAAAAAAAAtAAAAAAAAAAAAAAAAAAAAAAAAAtAtAAAAAtAAtAAAAAAAAAAAAAAAAAAAAtAAAtAAAAAAAAAAAAAAA',
       'AAAAAAAAAAAAAAAAtAAAtAAAtAttAAAAAAtAAAAAAAAAAAAAAAAAAtAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAtAAAAAAAAAAA',
       'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAtAAAAAAAAAAttAAAAtAAAAAtAAAAAAAAAAAAtAAAAAAAAttAAAAAAAAAAAAAAAAAAAAAAA',
       'AAAAAAAAAAAAAAAtAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAtAAAAtAAtAtAAAAAAAAAAAAtAAAtAAAtAAAAAAAAAAAAAA',
       'AAAAAAAAAAAAAtAAAAAAAAAAAAAAAAtAAAAAAAAtAAAAAAAAAAAAAAAAAAAAtAAtAAAAtAtAAAAAAAAAtAAAAAAAAAAAAAAAAAAA',
       'AAAAAAAAAAAAAAAAtAAAAAAAtAAAAAAAAAAAAAttAAAAAAAAAAAAAAAAAAAAAAAtAAAAAAAAAtAAAAAAAAAAAAAAttAAAAAAAAAA',
       'AAAAAAAAAAAAAAAAtAAAAAAtAAAAAAAAAAAAAAAAAAAAAAAAAAAtAtAAAAAAAAAAAtAAAAAAAAAAAAAtAAAAAtAtAAAAAAAAAAAA',
       'AAAAAAAAAAAAAAAAttAAAAAAtAAAtAAAAAAAAAAAAAAAAAAAAAAAAAAAtAAAAAAAAAAAAttAAAAAAAAAAAtAAAAAAAAAAAAAAAAA',
 

In [63]:
allowed_alph = np.array([[False, True, False, True],
                         [False, False, True, False],
                         [False, True, False, False],
                         [True, False, False, False]])

mutations_rand(testseq, 10, rate=0.1, site_start=10, site_end=90, 
    alph_type="DNA",
    allowed_alph=allowed_alph,
    number_fixed=True,
    keep_wildtype=True,
)

array(['AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA',
       'AAAAAAAAAAAAtcAAAAAAAAAAAAAAAAAcAAAAAAAAAAAAAAAAtAAAAAAcAAAAAAAAAAAAAAAAAAAcAAAAcAAAAAcAAAAAAAAAAAAA',
       'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAcAAAAAtAAAAAAAAAAAAAAAAAAAAAAAAtAAtcAAAAAAtAAAAAcAAAAAAtAAAAAAAAAAAA',
       'AAAAAAAAAAAAAAAAAAAAAAAAtAAAAAAAAAAAtAAAAAAAActtAtAAAAAAAAAAAAAAAAAAAAAAAAAAAcAAAAAAAAAAAtAAAAAAAAAA',
       'AAAAAAAAAAAAAAAAAAAAAAAtAAcAAAAcAAAAtAtAAAAAAAAAAAAAAAAAAAAtAAAAAAAAAAAAAAAAAAAcAAAtAAAAAAAAAAAAAAAA',
       'AAAAAAAAAAAAAAAtAAAAtAAAtAAAAAAAAcAAAAAAAAAAAAtAAAAAAAAAAtAAAAAAcAAAAAAAAAcAAAAAAAAAAAAAAAAAAAAAAAAA',
       'AAAAAAAAAAAAAAAAAAAAcAAAAAAAAAtAAAAAAAtAAActAAAAAAAAAAAAtAAAAAAAAAcAAAAcAAAAAAAAAAAAAAAAAAAAAAAAAAAA',
       'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAtAAAAAAAAAAAAAcAAAAAAAAAAccAAAAAAcAAAAAAAtAAAAtAAAAAtAAAAAAAAAAA',
       'AAAAAAAAAAAtAAAAAAAAAAAAtAAAAAAcAAAAAAAAAAAAAAAAAAAAAtccAAcAAAAAcAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA',
 

In [65]:
testseq = 'ACGT' * 25

allowed_alph = np.array([[False, False, False, True],
                         [False, False, True, False],
                         [False, True, False, False],
                         [True, False, False, False]])

mutations_rand(testseq, 10, rate=0.1, site_start=10, site_end=90, 
    alph_type="DNA",
    allowed_alph=allowed_alph,
    number_fixed=True,
    keep_wildtype=True,
)

array(['ACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGT',
       'ACGTACGTACGTACGTACGTACGaACGTACGTtCGTACGTACGTACcaACGTACGTAgGTACGTACGTACcTACGTACGTACcTAgGTACGTACGTACGT',
       'ACGTACGTACGTACGTACcTACGTACGTACGTACGTACGaACGTACGTACGTACGTACGaACGTACGTACGatCGTAgGTAgGTACcTACGTACGTACGT',
       'ACGTACGTACGTACGTAgGTACGatCcTACGTACcaACGTACGTACGTACGTACGTACGaACGTACGTACGTACGTACGTACGTAgGTACGTACGTACGT',
       'ACGTACGTACGTACGTACGTACGTACGaACGTACGTACGTACGTACGatCGTACGTtCGTACcTACcTtCGTACGTACGTACGTACcTACGTACGTACGT',
       'ACGTACGTACGaACGTACGTACGTACGTACGTtgGTACGTACGTtCcTACGTACGaACGTACGTACGTACGTACcTtCGTACGTACGTACGTACGTACGT',
       'ACGTACGTACGTACGTACcTACGTACGTACGTACcaACGTACGaACGTAgGTACGTACGTACGTACGaAgGTACGTACGTACcTACGTACGTACGTACGT',
       'ACGTACGTACGTACGTACGTACcTACGTACGTACGTAgGTACGTAgGaACGaACGTACGTACGTtCGTACcTACGTACGTACGTAgGTACGTACGTACGT',
       'ACGTACGTACGTACGTACGTACGaACGaACcTACGTACGTACGTACGTAgGTACGTACGTACGTACGTAgGTAgGTAgGTACGTACcTACGTACGTACGT',
 